In [1]:
import os
import glob

import numpy as np
import nibabel as nib
from tqdm import tqdm

In [13]:
def binarize_segmentation(img_np, only_specific_vessel_class: int = None):
    if not only_specific_vessel_class:
        img_np = np.where(img_np >= 1, 1, 0)
    else:
        img_np = np.where(img_np == only_specific_vessel_class, 1, 0)
    return img_np


In [14]:
def transform_label_vols_in_dir(filepaths: list, output_dir: str, transformation: callable):
    os.makedirs(output_dir, exist_ok=True)
    for file_path in tqdm(filepaths):
        img_name = os.path.basename(file_path)
        img = nib.load(file_path)
        img_np = img.get_fdata()
        img_np = transformation(img_np)
        img = nib.Nifti1Image(img_np, img.affine, img.header)
        nib.save(img, os.path.join(output_dir, img_name))

## Remap label data

In [19]:
data_dir = os.path.join('..', '..', '..', 'data', 'nnUNet_raw', 'Dataset005_ADAM3ClassesAneurysmsOnly')
data_dst_dir = os.path.join('..', '..', '..', 'data', 'nnUNet_raw', 'Dataset005_ADAMBinaryAneurysmsOnly')
os.makedirs(data_dst_dir, exist_ok=True)

labels_tr_dir_org = os.path.join(data_dir, 'labelsTr', '*.nii.gz')
labels_tr_dst_org = os.path.join(data_dst_dir, 'labelsTr')

labels_ts_dir_org = os.path.join(data_dir, 'labelsTs', '*.nii.gz')
labels_ts_dst_dir = os.path.join(data_dst_dir, 'labelsTs')

file_paths_train = glob.glob(labels_tr_dir_org)
file_paths_test = glob.glob(labels_ts_dir_org)

In [23]:
# Train
transform_label_vols_in_dir(
    file_paths_train, 
    labels_tr_dst_org,
    binarize_segmentation
)

# Test
transform_label_vols_in_dir(
    file_paths_test, 
    labels_ts_dst_dir,
    binarize_segmentation
)

100%|██████████| 23/23 [00:12<00:00,  1.87it/s]


In [24]:
train_dir_org = os.path.join(data_dir, 'imagesTr')
train_dir_dst = os.path.join(data_dst_dir, 'imagesTr')

In [25]:
!ln -s $train_dir_org $train_dir_dst

In [26]:

test_dir_org = os.path.join(data_dir, 'imagesTs')
test_dir_dst = os.path.join(data_dst_dir, 'imagesTs')

In [27]:
!ln -s $test_dir_org $test_dir_dst